In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange
from functools import partial
from JAxtar.bgpq import HashTableHeapValue, BGPQ

/tmp/ipykernel_2129239/3628738232.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
max_size = 100
group_size = 10
filled_sample = 8

In [4]:
heap = BGPQ.make_heap(max_size, group_size, HashTableHeapValue)
insert = jax.jit(heap.insert)
delete_mins = jax.jit(heap.delete_mins)

2024-07-30 08:52:10.859594: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [5]:
heap_key = jax.random.uniform(jax.random.PRNGKey(0), shape=(filled_sample,), minval=0, maxval=10)
value = jax.vmap(HashTableHeapValue.default)(jnp.arange(filled_sample))
make_batch = BGPQ.make_batched(heap_key, value, group_size)

In [6]:
for i in range(10):
    heap_key = jax.random.uniform(jax.random.PRNGKey(i), shape=(filled_sample,), minval=0, maxval=10)
    value = jax.vmap(HashTableHeapValue.default)(jnp.arange(filled_sample))
    block_key, block_val = BGPQ.make_batched(heap_key, value, group_size)
    heap, buffer_oveflow = insert(heap, block_key[0], block_val[0])
    print("----------------")
    for i in range(heap.max_size // heap.group_size):
        print(heap.key_store[i])
    print(heap.size)
    print(buffer_oveflow)
    print("----------------")

----------------
[1.0100961 1.953876  3.4965682 3.5358644 5.3222647 8.708868  9.524387
 9.829632        inf       inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[1]
False
----------------
----------------
[0.22907615 0.5912006  0.60886264 1.0100961  1.5803981  1.953876
 2.0243526  3.4965682  3.5358644  5.3222647 ]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf

In [7]:
size = int(heap.size[0]+1)
for i in range(size):
    heap, min_key, min_val = delete_mins(heap)

    print("----------------")
    print(min_key)
    print("----------------")
    for i in range(10):
        print(heap.key_store[i])
    print(heap.size)

----------------
[0.0197506  0.06212354 0.1193893  0.22907615 0.47122478 0.5544841
 0.5673051  0.5912006  0.60886264 0.752697  ]
----------------
[0.87839127 0.9131849  0.92179894 0.9417033  1.0100961  1.5803981
 1.953876   2.0243526  2.091118   2.2245538 ]
[2.4203205 2.581662  2.6958072 2.9146695 3.066901  3.1737351 3.2282853
 3.3229625 3.3524168 3.353783 ]
[5.162902  5.3222647 8.16434   8.375758  8.551788  8.579488  8.708868
 9.418657  9.524387  9.574013 ]
[3.3943057 3.4965682 3.5358644 6.8252063 7.7763596 8.780643  9.261496
 9.364391  9.936373  9.992442 ]
[4.5358324 4.6061287 4.868312  5.493932  6.492506  7.5331354 8.1164055
 9.349147  9.623257  9.829632 ]
[4.1012216 4.240223  4.268917  4.3381405 4.6185637 5.084324  6.087223
 6.2697735 6.992921  7.1907115]
[3.757888  3.7924135 3.988229  4.05226   4.2749906 4.377426  8.627738
 8.668605  8.688327  8.733465 ]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[inf inf inf inf inf inf inf inf inf inf]
[7